<a href="https://colab.research.google.com/github/SohaHussain/LangChain/blob/main/Document_Splitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Document Splitting


Document splitting happens after you load your data
into the document format and before
it goes into the vector store.

The basis
of all the text splitters in Lang Chain involves splitting on
chunks in some chunk size with
some chunk overlap. A chunk overlap is generally kept as a
little overlap between two chunks, like a sliding window as we
move from one to the other. And this allows for
the same piece of context to be at the end of one chunk and at the
start of the other and helps create some
notion of consistency.

In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00


In [3]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.5 MB/s eta 0:00:00


In [23]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.6/272.6 kB 4.5 MB/s eta 0:00:00


In [30]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.2 MB/s eta 0:00:00


In [43]:
import openai
import langchain
openai_key = '##'

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [8]:
chunk_size =26
chunk_overlap = 4

In [9]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [11]:
text1 = 'abcdefghijklmnopqrstuvwxyz'
r_splitter.split_text(text1) # gives the same string as chunk size is 26

['abcdefghijklmnopqrstuvwxyz']

In [13]:
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'
r_splitter.split_text(text2) # last 4 letters are repeated because chunk overlap is 4

['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

###Recursive Splitting

In [14]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

In [15]:
len(some_text)

496

In [17]:
c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator = ' '
)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separators=["\n\n", "\n", " ", ""] # What this mean is that when you're splitting a piece of text it will first try to split it by double newlines.
                                       # And then, if it still needs to split the individual chunks more it will go on to single newlines. And then, if it
                                       # still needs to do more it goes on to the space. And then, finally it will just go character by character if it really needs to do that.
)

In [19]:
c_splitter.split_text(some_text) # character text splitter splits on spaces. And so, we end up with the weird separation in the middle of the sentence

['When writing documents, writers will use document structure to group content. This can convey to the reader, which idea\'s are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also,',
 'have a space.and words are separated by space.']

In [20]:
r_splitter.split_text(some_text) # The recursive text splitter first tries to split on double newlines, and so here it splits it up into two paragraphs.
                                 # Even though the first one is shorter than the 450 characters, we specified this is probably a better split because now the two
                                 # paragraphs that are each their own paragraphs are in the chunks as opposed to being split in the middle of a sentence.

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.",
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [21]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150, # reducing the chunk size
    chunk_overlap=0,
    separators=["\n\n", "\n", "\. ", " ", ""]
)
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example,",
 'closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this',
 'string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [24]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/MachineLearning-Lecture01.pdf")
pages = loader.load()

In [25]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

In [26]:
docs = text_splitter.split_documents(pages)
len(docs)

77

In [27]:
len(pages)

22

### Token Splitting

The reason that this is useful is because often LLMs have context
windows that are designated by token count. And so,
it's important to know what the tokens are, and
where they appear. And then, we can split on them to have a slightly
more representative idea of how the LLM would view them.

In [28]:
from langchain.text_splitter import TokenTextSplitter

In [31]:
text_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)

In [32]:
text_splitter.split_text(text1)

['abc',
 'def',
 'gh',
 'ij',
 'kl',
 'mn',
 'op',
 'q',
 'r',
 'st',
 'uv',
 'w',
 'xy',
 'z']

In [34]:
text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0)
docs = text_splitter.split_documents(pages)
docs[0]

Document(page_content='MachineLearning-Lecture01  \n', metadata={'source': '/content/MachineLearning-Lecture01.pdf', 'page': 0})

In [35]:
pages[0].metadata

{'source': '/content/MachineLearning-Lecture01.pdf', 'page': 0}

In [36]:
docs[1]

Document(page_content='Instructor (Andrew Ng):  Okay. Good', metadata={'source': '/content/MachineLearning-Lecture01.pdf', 'page': 0})

###Context Aware Splitting

Chunking aims to keep text with common context together.

A text splitting often uses sentences or other delimiters to keep related text together but many documents (such as Markdown) have structure (headers) that can be explicitly used in splitting.

We can use MarkdownHeaderTextSplitter to preserve header metadata in our chunks.

In [37]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [38]:
markdown_document = """# Title\n\n \
## Chapter 1\n\n \
Hi this is Jim\n\n Hi this is Joe\n\n \
### Section \n\n \
Hi this is Lance \n\n
## Chapter 2\n\n \
Hi this is Molly"""

In [39]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [40]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)
md_header_splits = markdown_splitter.split_text(markdown_document)

In [41]:
md_header_splits[0]

Document(page_content='Hi this is Jim  \nHi this is Joe', metadata={'Header 1': 'Title', 'Header 2': 'Chapter 1'})

In [42]:
md_header_splits[1]

Document(page_content='Hi this is Lance', metadata={'Header 1': 'Title', 'Header 2': 'Chapter 1', 'Header 3': 'Section'})